In [2]:
%pip install ruprompts

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 86.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 104.7 MB/s eta 0:00:00


In this tutorial we show how to load and use the pretrained prompts from HuggingFace Hub.

In [3]:
# although not used directly, this import is necessary,
# since it adds our custom pipelines to transformers
import ruprompts

from transformers import pipeline

Let's first load the joke prompt:

In [4]:
ppln_joke = pipeline("text-generation-with-prompt", prompt="konodyuk/prompt_rugpt3large_joke", device=0)

No model was supplied, defaulted to sberbank-ai/rugpt3large_based_on_gpt2 and revision aa2b602 (https://huggingface.co/sberbank-ai/rugpt3large_based_on_gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


/usr/local/lib/python3.9/dist-packages/transformers/models/auto/modeling_auto.py:1295: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Note that we didn't specify the `model=` and `tokenizer=` arguments. They are inferred from pretrained prompt config.

Using the pipeline with prompt is almost identical to using the regular text generation pipeline:

In [5]:
ppln_joke("Говорит как-то дуб вороне")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': 'Говорит как-то дуб вороне: "Посмотри, эта воронья масть плохая, а та хорошая!"<pad>'}]

Although it is more convenient to omit model arguments, it leads to reinstantiation of the same model each time we create a pipeline. An easy fix is to create model and tokenizer once and then pass it to all the pipelines:

In [6]:
from transformers import GPT2LMHeadModel, AutoTokenizer

model_id = "sberbank-ai/rugpt3large_based_on_gpt2"
model = GPT2LMHeadModel.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


It will speed up loading the proverb pipeline:

In [7]:
ppln_proverb = pipeline("text-generation-with-prompt", prompt="konodyuk/prompt_rugpt3large_proverb", model=model, tokenizer=tokenizer, device=0)
ppln_proverb("Сколько ни")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': 'Сколько ни говорить, а ничего доброго не скажешь.<pad>'}]

Loading and using text2text prompts is almost identical:

In [8]:
ppln_detox = pipeline("text2text-generation-with-prompt", prompt="konodyuk/prompt_rugpt3large_detox_russe", model=model, tokenizer=tokenizer, device=0)
ppln_detox({"toxic_comment": "Ублюдок, мать твою, а ну иди сюда"})

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': 'А ну вернись<pad>'}]

In [9]:
long_text = """Несмотря на споры, большинство учёных сошлись во мнении, что кошка является полуодомашненным животным, то есть она способна на сосуществование с человеком, но, потеряв с ним контакт, легко возвращаются к дикому образу существования. Хотя у кошки наблюдаются генетические изменения в сравнении с диким предком, эта разница в 10 раз меньше, чем у собак с волками. Учёные считают, что дикая кошка действительно могла сама прийти к человеку, чтобы питаться грызунами, а такие отношения характеризовались как соседские, и уже через несколько тысяч лет люди сами стали одомашнивать маленьких хищников. Это также, вероятно, объясняет, почему модель поведения кошки почти не изменилась; при одомашнивании собаки из волка человек изменил её образ жизни и среду обитания, кошка же претерпела минимальные изменения Кошка сумела сохранить модель поведения, присущую её диким предкам. Она почти так же хорошо охотится, как дикая кошка, но в то же время способна мирно сосуществовать с человеком, проявлять к нему эмоциональную привязанность, нежность или даже выказывать игривое поведение."""
ppln_summary = pipeline("text2text-generation-with-prompt", prompt="konodyuk/prompt_rugpt3large_summarization_mlsum", model=model, tokenizer=tokenizer, device=0)

ppln_summary(long_text, num_beams=2, num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': 'Вероятнее всего, кошка стала одомашненной из-за того, что потеряла с человеком контакт<pad><pad><pad><pad><pad><pad>'},
 {'generated_text': 'Учёные уверены, что дикая кошка могла сама прийти к человеку, чтобы питаться грызунами<pad><pad><pad><pad><pad><pad><pad><pad>'},
 {'generated_text': 'Дикие кошки, которые не могли найти общий язык с человеком, стали более дружелюбными<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'},
 {'generated_text': 'Дикая кошка могла прийти к человеку, чтобы питаться грызунами, а такие отношения характеризовались как соседские<pad><pad><pad><pad><pad>'},
 {'generated_text': 'Учёные пришли к выводу, что дикие кошки, которые выжили в суровых условиях Арктики, могли адаптироваться к человеку<pad>'}]

In [10]:
ppln_summary = pipeline("text2text-generation-with-prompt", prompt="konodyuk/prompt_rugpt3large_title_mlsum", model=model, tokenizer=tokenizer, device=0)
ppln_summary(long_text, num_beams=2, num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': 'Кошка-полумаска<pad><pad><pad>'},
 {'generated_text': 'Дикая кошка вернулась к человеку<pad><pad><pad>'},
 {'generated_text': 'Кошка и человек сохранили модель поведения<pad><pad>'},
 {'generated_text': 'Кошка-человек<pad><pad><pad><pad><pad>'},
 {'generated_text': 'Дикая кошка и человек на одной планете<pad>'}]

In some cases pipeline takes multiple arguments:

In [11]:
context = """В 1997 году Шмидхубер и Сепп Хохрайтер опубликовали работу, описывающую рекуррентную нейронную сеть, которую авторы назвали «Долгая краткосрочная память». В 2015 году эта архитектура была использована в новой реализации распознавания речи в программном обеспечении компании Google для смартфонов.

Исследования Шмидхубера также включают в себя генерализации колмогоровской сложности и метрики «скорость важна» (Speed Prior), создание концепции Машины Гёделя.

В 2014 году Шмидхубер основал компанию Nnaisense для работы в сфере коммерческого применения технологий искусственного интеллекта в таких областях как финансы, тяжёлая промышленность и самоуправляемый автотранспорт. Сепп Хохрайтер и Яан Таллинн занимают в компании пост советников."""

ppln_qa = pipeline("text2text-generation-with-prompt", prompt="konodyuk/prompt_rugpt3large_qa_sberquad", model=model, tokenizer=tokenizer, device=0)
ppln_qa({"context": context, "question": "С кем Шмидхубер опубликовал работу?"})

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': 'Сепп Хохрайтер<pad>'}]

So far we needed to create a separate pipeline for each task. It may be inconvenient if you want to serve multiple prompts simultaneously. For this case we provide a more low-level interface, called MultiPrompt:

In [12]:
from ruprompts import MultiPrompt

mp = MultiPrompt({
    "detox": "konodyuk/prompt_rugpt3large_detox_russe",
    "qa": "konodyuk/prompt_rugpt3large_qa_sberquad",
})
mp.patch(model, tokenizer)


Now we need to create a regular generation pipeline:

In [13]:
ppln = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0, return_full_text=False, do_sample=True, eos_token_id=0, max_new_tokens=100)

To switch the task, we just need to change the key:

In [14]:
ppln(mp(key="detox", toxic_comment="Какого черта ты сюда припёрся?"))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': 'Почему ты сюда пришел<pad>'}]

In [15]:
ppln(mp(key="qa", context=context, question="В каком году была основана Nnaisense?"))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': 'В 2014 году<pad>'}]